In [11]:
from models.yolov8Face.main import YOLOv8_face
labels = {0: "angry", 1: "disgust", 2:"fear", 3:"happy", 4:"sad", 5:"surprise", 6:"neutral"}


In [12]:
from deepface.extendedmodels import Emotion
senti_model = Emotion.loadModel()

In [13]:
senti_model.save("senti_base")

INFO:tensorflow:Assets written to: senti_base/assets


INFO:tensorflow:Assets written to: senti_base/assets


In [14]:
import cv2 
import numpy as np
def draw(image, boxes, scores, id=1):
    for box, score in zip(boxes, scores):
        x, y, w, h = box.astype(int)
        # Draw rectangle
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), thickness=2)

        cropped_image = image[y:y+h, x:x+w]
        cropped_image = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        cropped_image = cv2.resize(cropped_image, (48,48))
        result = senti_model.predict(np.expand_dims(cropped_image, axis=0), verbose=False)
        emotion = np.argmax(result)
        image = cv2.putText(image, f"{labels[emotion]}", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), thickness=1)


    return image 

In [15]:
YOLOv8_face_detector = YOLOv8_face('models/yolov8Face/weights/yolov8n-face.onnx', conf_thres=0.4)

In [ ]:
# Open the video file
video_path = "experimentation_videos/videoplayback.mp4"
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('model_output/output1.avi',cv2.VideoWriter_fourcc('M','J','P','G'), fps, (frame_width,frame_height))

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:
        # Detect Objects
        boxes, scores, classids, kpts = YOLOv8_face_detector.detect(frame)
        # annotated_frame = draw(frame, boxes, frame_width, frame_height, id=1)
        # Draw detections
        dstimg = YOLOv8_face_detector.draw_detections(frame, boxes, scores, kpts)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", dstimg)
    else:
        break

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release() # Release the video file
cv2.destroyAllWindows()
